In [1]:
import sys
sys.path.append('../')

import numpy
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from keras.layers import SimpleRNN
from keras.models import Sequential

from testboard.data_mining.helpers.stock_util import StockUtil
from testboard.data_mining.smote import duplicate_data
from testboard.models.densegru import DenseGRU
from testboard.models.denselstm import DenseLSTM

Using TensorFlow backend.


### Getting stock data

In [12]:
stockutil = StockUtil(['PETR3'], [9])
prices, preds = stockutil.average_prices_preds()

[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


### Slice dataset in training data and test data

In [16]:
train_proportion = 0.7

train_size = int(len(prices[0]) * train_proportion)

train_x = prices[0][0:train_size]
test_x = prices[0][train_size:len(prices[0])]
train_y = preds[0][0:train_size]
test_y = preds[0][train_size:len(prices[0])]

### Testing on KNN

In [15]:
X = numpy.array(train_x).reshape(len(train_x), 1)
print(X)
Y = train_y

neigh = KNeighborsRegressor(n_neighbors=5)
neigh.fit(X, Y)
X_test = numpy.array(test_x).reshape(len(test_x), 1)

print(neigh.predict(X_test))
score = 0
for i, pred in enumerate(neigh.predict(X_test)):
    if (pred >= 0.5 and test_y[i] is 1) or (pred < 0.5 and test_y[i] is 0):
        score += 1
        
print("accuracy: " + str(score/len(test_y)))

[[15.445     ]
 [15.35444444]
 [15.26777778]
 [15.21944444]
 [15.16333333]
 [15.07111111]
 [14.98944444]
 [14.93166667]
 [14.87      ]
 [14.83388889]
 [14.84666667]
 [14.85944444]
 [14.83777778]
 [14.79888889]
 [14.77111111]
 [14.74277778]
 [14.70055556]
 [14.66444444]
 [14.605     ]
 [14.54388889]
 [14.53      ]
 [14.52      ]
 [14.49666667]
 [14.45833333]
 [14.41666667]
 [14.38888889]
 [14.355     ]
 [14.31833333]
 [14.25722222]
 [14.15666667]
 [14.06      ]
 [13.97611111]
 [13.90666667]
 [13.865     ]
 [13.83388889]
 [13.76888889]
 [13.66666667]
 [13.57388889]
 [13.50555556]
 [13.435     ]
 [13.35333333]
 [13.29666667]
 [13.25166667]
 [13.21222222]
 [13.22277778]
 [13.27555556]
 [13.32388889]
 [13.38555556]
 [13.48722222]
 [13.60444444]
 [13.70111111]
 [13.76333333]
 [13.79722222]
 [13.81888889]
 [13.83611111]
 [13.83388889]
 [13.79444444]
 [13.72222222]
 [13.64166667]
 [13.56888889]
 [13.51777778]
 [13.48      ]
 [13.43944444]
 [13.40055556]
 [13.37833333]
 [13.38388889]
 [13.41277

### Testing on LSTM

In [5]:
lstm = DenseLSTM(look_back=6, dense=False)
lstm.create_data_for_fit(numpy.array(prices[0]).reshape(len(prices[0]), 1))
result = lstm.fit_and_evaluate(batch_size=1, epochs=5000)
print("accuracy: " + str(result['acc']))

63/63 [==============================] - 0s 1ms/step
[NeuralNetwork] Test Accuracy:0.8253968358039856
accuracy: 0.8253968358039856


### Testing on LSTM optimized

In [6]:
lstm = DenseLSTM(look_back=6, dense=1, lstm_cells=50)
lstm.create_data_for_fit(numpy.array(prices[0]).reshape(len(prices[0]), 1))
result = lstm.fit_and_evaluate(batch_size=1, epochs=5000)
print("accuracy: " + str(result['acc']))

63/63 [==============================] - 0s 2ms/step
[NeuralNetwork] Test Accuracy:0.8571428656578064
accuracy: 0.8571428656578064


### Testing on GRU

In [7]:
gru = DenseGRU(look_back=6, dense=False)
gru.create_data_for_fit(numpy.array(prices[0]).reshape(len(prices[0]), 1))
result = gru.fit_and_evaluate(batch_size=1, epochs=5000)
print("accuracy: " + str(result['acc']))

63/63 [==============================] - 0s 1ms/step
[NeuralNetwork] Test Accuracy:0.8571428656578064
accuracy: 0.8571428656578064


### Testing on SVM

In [8]:
svr = SVR(kernel='linear', C=1, epsilon=0.2)
svr.fit(X, Y)
X_test = numpy.array(test_x).reshape(len(test_x), 1)
svr.predict(X_test)

array([0.82893668, 0.84642084, 0.85657923, 0.86341661, 0.87259823,
       0.87953329, 0.87972864, 0.8761146 , 0.8719145 , 0.87621228,
       0.88324501, 0.88891027, 0.89486856, 0.89545462, 0.89125451,
       0.88676138, 0.87689601, 0.85843509, 0.83929043, 0.82522496,
       0.81506657, 0.80715474, 0.80090342, 0.79572655, 0.7929916 ,
       0.79777776, 0.80471282, 0.80588494, 0.80627565, 0.80705707,
       0.80373605, 0.79689867, 0.78488442, 0.76632582, 0.74522762,
       0.72696205, 0.71524082, 0.70508243, 0.69150535, 0.67900271,
       0.66698846, 0.6571231 , 0.66366745, 0.68261676, 0.70459405,
       0.7258876 , 0.74688812, 0.77257714, 0.80520121, 0.84534639,
       0.88988704, 0.92456232, 0.94546516, 0.96197255, 0.97291236,
       0.97789388, 0.98639177, 0.99615945, 0.99752693, 0.99361985,
       0.98492661, 0.97535428, 0.96343771, 0.95034901, 0.93511142,
       0.90825028, 0.87494248, 0.84485801, 0.81682475])

### Testing on RNN

In [9]:
model = Sequential()

model.add(SimpleRNN(activation="sigmoid", units=1))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(gru.train_x, gru.train_y, epochs=5000, verbose=0)
result = model.evaluate(gru.test_x, gru.test_y)
print('accuracy: ' + str(result[1]))

63/63 [==============================] - 0s 3ms/step
accuracy: 0.4920634925365448
